In [2]:
import boto3

bucket_list = []

s3 = boto3.resource('s3')
s3_object = s3.Bucket('citibikejsonfiles')

for file in s3_object.objects.filter(Prefix = 'dataStationInformation'):
    file_name = file.key
    if file_name.find(".json") != -1:
        bucket_list.append(file.key)

len_bucket_list = print(len(bucket_list))
print(bucket_list[0:10])


731
['dataStationInformation 2023-01-14 01:40:37.364932.json', 'dataStationInformation 2023-01-14 01:50:37.351757.json', 'dataStationInformation 2023-01-14 02:00:37.354085.json', 'dataStationInformation 2023-01-14 02:10:37.297757.json', 'dataStationInformation 2023-01-14 02:20:37.491889.json', 'dataStationInformation 2023-01-14 02:30:37.376058.json', 'dataStationInformation 2023-01-14 02:40:37.058998.json', 'dataStationInformation 2023-01-14 02:50:37.666289.json', 'dataStationInformation 2023-01-14 03:00:37.288828.json', 'dataStationInformation 2023-01-14 03:10:37.558505.json']


In [8]:
from io import StringIO
import io
import pandas as pd

dataframe = [] # empty list of dataframes

for file in bucket_list:
    s3_obj = s3.Object('citibikejsonfiles', file)
    data = s3_obj.get()['Body'].read()
    dataframe.append(pd.read_json(io.BytesIO(data)))

In [15]:
len(dataframe)

731

In [17]:
dataframe[0]

,data,last_updated,ttl
stations,"[{'eightd_has_key_dispenser': False, 'short_na...",1673660428,5


In [22]:
dataStationInformation = pd.json_normalize(dataframe[730]['data']['stations'])
dataStationInformation

,eightd_station_services,station_id,station_type,has_kiosk,eightd_has_key_dispenser,legacy_id,region_id,external_id,capacity,short_name,name,lat,rental_methods,electric_bike_surcharge_waiver,lon,rental_uris.ios,rental_uris.android
0,[],66dd5960-0aca-11e7-82f6-3863bb44ef7c,classic,True,False,3185,70,66dd5960-0aca-11e7-82f6-3863bb44ef7c,22,JC003,City Hall,40.717732,"[KEY, CREDITCARD]",False,-74.043845,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan
1,[],66dd58f2-0aca-11e7-82f6-3863bb44ef7c,classic,True,False,3184,70,66dd58f2-0aca-11e7-82f6-3863bb44ef7c,14,JC002,Paulus Hook,40.714145,"[KEY, CREDITCARD]",False,-74.033552,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan
2,[],66dd7b37-0aca-11e7-82f6-3863bb44ef7c,classic,True,False,3203,70,66dd7b37-0aca-11e7-82f6-3863bb44ef7c,26,JC009,Hamilton Park,40.727596,"[KEY, CREDITCARD]",False,-74.044247,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan
3,[],66dd6441-0aca-11e7-82f6-3863bb44ef7c,classic,True,False,3199,70,66dd6441-0aca-11e7-82f6-3863bb44ef7c,14,JC008,Newport Pkwy,40.728745,"[KEY, CREDITCARD]",False,-74.032108,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan
4,[],66dd5a42-0aca-11e7-82f6-3863bb44ef7c,classic,True,False,3187,70,66dd5a42-0aca-11e7-82f6-3863bb44ef7c,22,JC006,Warren St,40.721124,"[KEY, CREDITCARD]",False,-74.038051,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1830,[],5014,classic,True,False,5014,71,81640d04-c36a-4fb1-98b0-85be10be516c,21,4080.01,Schenectady Ave & Pacific St,40.676860,"[CREDITCARD, KEY]",False,-73.933270,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan
1831,[],5020,classic,True,False,5020,71,c7dbc7b5-240b-494d-8663-dbee4103a20c,21,3934.07,Troy Ave & Park Pl,40.672510,"[CREDITCARD, KEY]",False,-73.936420,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan
1832,[],5021,classic,False,False,5021,71,1789e458-c466-4d77-aba6-365f5c3823ed,0,4196.01,Lewis Ave & Fulton St,40.679720,"[CREDITCARD, KEY]",False,-73.934670,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan
1833,[],5022,classic,True,False,5022,70,2290031f-e7d8-40a3-9ad6-566df493bbbc,42,JC115,Grove St PATH,40.719410,"[CREDITCARD, KEY]",False,-74.043090,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan


In [24]:
vc = dataStationInformation.station_id.duplicated()
vc.value_counts()

False    1835
Name: station_id, dtype: int64

In [23]:
dataStationInformation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1835 entries, 0 to 1834
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   eightd_station_services         1835 non-null   object 
 1   station_id                      1835 non-null   object 
 2   station_type                    1835 non-null   object 
 3   has_kiosk                       1835 non-null   bool   
 4   eightd_has_key_dispenser        1835 non-null   bool   
 5   legacy_id                       1835 non-null   object 
 6   region_id                       1834 non-null   object 
 7   external_id                     1835 non-null   object 
 8   capacity                        1835 non-null   int64  
 9   short_name                      1835 non-null   object 
 10  name                            1835 non-null   object 
 11  lat                             1835 non-null   float64
 12  rental_methods                  18